In [100]:
import os
import os.path as osp
import sys
import h5py
import numpy as np
sys.path.append("/data/lyx/software/StereoMMv1/")
from process_img import *
from utils import *
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class ImageDataset(Dataset):
    def __init__(self, data):
        self.data = data  # 假设data已经是处理好的张量

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
def save_img_feat(data,save_path,feat_file):
    print('shape of image_feature：',data.shape)
    data.to_pickle(os.path.join(save_path,feat_file))
    
Image.MAX_IMAGE_PIXELS = None
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [54]:
bin_size = 100
crop_size = 150
output='hest_data/image_out'
name = "TENX94"
name = "".join(name)
label_txt_file=str(name+'.h5')
adata_file=str(name+'.h5ad')
tile_path = os.path.join(output,name)

In [55]:
current_path = "/data/lyx/software/StereoMMv1/"
data_path = "/data/lyx/hubs/Cpath/hest_data/"

In [56]:
osp.join(data_path,label_txt_file)

'/data/lyx/hubs/Cpath/hest_data/TENX94.h5'

In [39]:
with h5py.File(osp.join(data_path,label_txt_file), 'r') as f:
# 遍历文件中的所有组和数据集
    for key in f.keys():
        print(f[key], key, f[key].name)

<HDF5 dataset "barcode": shape (6078, 1), type "|O"> barcode /barcode
<HDF5 dataset "coords": shape (6078, 2), type "<i8"> coords /coords
<HDF5 dataset "img": shape (6078, 224, 224, 3), type "|u1"> img /img


In [78]:
with h5py.File(osp.join(data_path,label_txt_file), "r") as f:
    # 获取数据集对象
    img_dataset = f["img"]
    
    # 将数据集读取为NumPy数组
    img_data = img_dataset[:]  # 或 np.array(img_dataset)
    
print("数据形状:", img_data.shape)  # 应为 (6078, 224, 224, 3)
print("数据类型:", img_data.dtype)  # 应为 uint8

数据形状: (6078, 224, 224, 3)
数据类型: uint8


In [57]:
# with h5py.File(osp.join(data_path,label_txt_file), "r") as f:
#     img_dataset = f["img"]
#     chunk_size = 100  # 每批读取100张图像
    
#     for i in range(0, img_dataset.shape[0], chunk_size):
#         batch = img_dataset[i:i+chunk_size, :, :, :]
#         print(f"已读取批次 {i//chunk_size}: 形状={batch.shape}")
#         # 在此处理batch数据

In [68]:
model_com = models.resnet50(pretrained=False)
pth_path = os.path.join(current_path, "torch_pths/resnet50-19c8e357.pth")
model_com.load_state_dict(torch.load(pth_path))
num_features = model_com.fc.in_features

In [69]:
### strip the last layer
model = torch.nn.Sequential(*list(model_com.children())[:-1])

In [60]:
#Linear(in_features=2048, out_features=1024, bias=True)

In [61]:
# import timm
# feature_extractor = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)#embed_layer=ConvStem
# feature_extractor.head = nn.Identity()
# pth_path = os.path.join(current_path, "torch_pths/CHIEF_CTransPath.pth")

In [62]:
# def load_model(current_path,model_name = 'resnet50'):
#     if model_name == 'resnet50':
#         model_com = models.resnet50(pretrained=False)
#         if 'StereoMMv1' in current_path:
#             pth_path = os.path.join(current_path, "torch_pths/resnet50-19c8e357.pth")
#         else:
#             pth_path = os.path.join(current_path, "StereoMMv1/torch_pths/resnet50-19c8e357.pth")
#         model_com.load_state_dict(torch.load(pth_path))
#         num_features = model_com.fc.in_features
#         ### strip the last layer
#         feature_extractor = torch.nn.Sequential(*list(model_com.children())[:-1])
#     elif model_name == 'CHIEF':
#         import timm
#         feature_extractor = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)#embed_layer=ConvStem
#         feature_extractor.head = nn.Identity()
#         if 'StereoMMv1' in current_path:
#             pth_path = os.path.join(current_path, "torch_pths/CHIEF_CTransPath.pth")
#         else:
#             pth_path = os.path.join(current_path, "StereoMMv1/torch_pths/CHIEF_CTransPath.pth")
#         td = torch.load(pth_path)
#         feature_extractor.load_state_dict(td['model'], strict=True)

#     feature_extractor.to(device)
#     feature_extractor.eval()
#     return(feature_extractor)

In [79]:
img_data = torch.from_numpy(img_data).permute(0, 3, 1, 2).float()  # 转换为浮点型
print(img_data.shape)
# 可选：归一化到 [0, 1] 范围（如果原始数据是 uint8 的 0-255）
# tensor_data = tensor_data / 255.0

torch.Size([6078, 3, 224, 224])


In [80]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

class ImageDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image  # 仅返回图像，无标签

# 定义数据增强（根据任务选择）
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [81]:
dataset = ImageDataset(
    images=img_data,
    transform=train_transform
)

In [77]:
model = model.to(device)

In [96]:
batch_size = 1
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [97]:
feat_outputs = []
model.eval()
for batch in loader:
    inputs = batch.to(device)
    outputs = model(inputs)
    outputs = outputs.data.cpu().numpy().ravel()
    feat_outputs.append(outputs)

In [105]:
img_emb = np.vstack(feat_outputs)

In [110]:
# save_img_feat(img_emb,output,feat_file = 'img_feat.pkl')
np.save(f"{name}_resnet_img_emb.npy",img_emb)
print('Features saved')

Features saved


In [ ]:
# 编码器定义
class ImageEncoder(nn.Module):
    def __init__(self,img_model):
        super().__init__()
        self.model = img_model
        self.pooling = nn.AdaptiveAvgPool1d(1024)
        self.fc = nn.Linear(in_features=2048, out_features=1024, bias=True)
    def forward(self, x):
        x = self.model(x)
        x = torch.cat(self.pooling(x),self.fc(x),axis=1)
        return F.normalize(x, p=2, dim=1)

In [ ]:
class TranscriptomeEncoder(nn.Module):
    def __init__(self, input_dim,pretrainmodel):
        super().__init__()
        self.token_emb = pretrainmodel.token_emb
        self.pos_emb = pretrainmodel.pos_emb
        self.encoder = pretrainmodel.encoder
        self.fc1 = nn.Linear(in_features=input_dim*2, out_features=1024, bias=True)
        self.fc2 = nn.Linear(in_features=input_dim*2, out_features=1024, bias=True)  
    def forward(self, x,position_gene_ids,x_padding):
        x = self.token_emb(x, output_weight = 0)
        position_emb = self.pos_emb(position_gene_ids)
        x += position_emb
        geneemb = self.encoder(x,x_padding)
        geneemb1 = geneemb[:,-1,:]
        geneemb2 = geneemb[:,-2,:]
        geneemb3, _ = torch.max(geneemb[:,:-2,:], dim=1)
        geneemb4 = torch.mean(geneemb[:,:-2,:], dim=1)
        x1 = torch.concat([geneemb1,geneemb2],axis=1)
        x2 = torch.concat([geneemb3,geneemb4],axis=1)
        x = torch.concat([self.fc1(x1),self.fc1(x2)],axis=1)
        return F.normalize(x, p=2, dim=1)

In [ ]:
# tgthighres = 'a5'
# geneemb_list = []
# for i in tqdm(range(gexpr_feature.shape[0])):
#     with torch.no_grad():
#         tmpdata = (np.log1p(gexpr_feature.iloc[i,:]/(gexpr_feature.iloc[i,:].sum())*1e4)).tolist()
#         totalcount = gexpr_feature.iloc[i,:].sum()
#         pretrain_gene_x = torch.tensor(tmpdata+[np.log10(totalcount)+float(tgthighres[1:]),np.log10(totalcount)]).unsqueeze(0)
#         data_gene_ids = torch.arange(19266, device=pretrain_gene_x.device).repeat(pretrain_gene_x.shape[0], 1)
        
#         value_labels = pretrain_gene_x > 0
#         x, x_padding = gatherData(pretrain_gene_x, value_labels, pretrainconfig['pad_token_id'])

        # position_gene_ids, _ = gatherData(data_gene_ids, value_labels, pretrainconfig['pad_token_id'])
        
        x = pretrainmodel.token_emb(torch.unsqueeze(x, 2).float().to(device), output_weight = 0)
        position_emb = pretrainmodel.pos_emb(position_gene_ids.to(device))
        x += position_emb
        geneemb = pretrainmodel.encoder(x,x_padding.to(device))
        geneemb_list.append(geneemb.cpu().numpy())

In [ ]:
# 初始化模型与参数
image_encoder = ImageEncoder()
tx_encoder = TranscriptomeEncoder(input_dim=20000)
logit_scale = nn.Parameter(torch.tensor([1.0]))

# 优化器
optimizer = torch.optim.AdamW([
    {'params': image_encoder.parameters()},
    {'params': tx_encoder.parameters()},
    {'params': [logit_scale], 'lr': 0.001}
], lr=0.0001)

# 训练循环
for images, tx_data in dataloader:
    img_emb = image_encoder(images)
    tx_emb = tx_encoder(tx_data)
    
    # 计算相似度
    logits = img_emb @ tx_emb.T * logit_scale.exp()
    labels = torch.arange(len(images), device=images.device)
    
    # 对称损失
    loss = (F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels)) / 2
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()